In [81]:
import sys
import cmath
import math
import os
import h5py

import matplotlib.pyplot as plt   # plots
import numpy as np
%matplotlib inline 


if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
import util.libutil as comn

from libra_py import units
from libra_py.data_visualize import colors, clrs_index
import libra_py.dynamics_plotting as dynamics_plotting
import libra_py.scan as scan
import libra_py.psi4_methods as psi4_methods
import libra_py.DFTB_methods as DFTB_methods

import py3Dmol   # molecular visualization

In [37]:
import dftpy
from dftpy.interface import ConfigParser, OptimizeDensityConf
from dftpy.config import DefaultOption, OptionFormat

In [82]:
class tmp:
    pass

def run_dftpy_adi(q, params_, full_id, dftpy_config):
    """
   
    This function executes the Psi4 quantum chemistry calculations and 
    returns the key properties needed for dynamical calculations.

    Args: 
        q ( MATRIX(ndof, ntraj) ): coordinates of the particle [ in Bohr units ]
        params ( dictionary ): model parameters
 
            * **params["labels"]** ( list of strings ): the labels of atomic symbolc - for all atoms,
                and in a order that is consistent with the coordinates (in triples) stored in `q`.
                The number of this labels is `natoms`, such that `ndof` = 3 * `natoms`. [ Required ]
            * **params["nstates"]** ( int ): the total number of electronic states 
                in this model [ default: 1 - just the ground state ]
            * **params["grad_method_gs"]** ( string ):  the name of the methodology to compute the 
                energy and gradients on the ground state [ defaut: "ccsd/sto-3g" ]
                Examples: 
                  "pbe/sto-3g", "mp2/aug-cc-pVDZ", "ccsd/aug-cc-pVDZ" # ground state energies, gradients                   
            * **params["grad_method_ex"]** ( string ):  the name of the methodology to compute the 
                energy and gradients on the excited states [ defaut: "eom-ccsd/sto-3g" ]
                Examples:                                     
                  "eom-ccsd/aug-cc-pVDZ", # excited state energies, gradients                  
                If you need just excited state energies (but not gradients), consider: 
                "cisd/aug-cc-pVDZ", adc/aug-cc-pVDZ                
            * **params["charge"]** ( int ): the total charge of the system [ default: 0 ]
            * **params["spin_multiplicity"]** ( int ): the total spin multiplicity [ default: 1 - singlet ]
            * **params["options"]** ( dictionary ): additional parameters of calculations [ default: empty ]
                Examples: 
                  - {} - noting extra
                  - {'reference':'rohf'}, 
                  - {'roots_per_irrep':[3, 0, 0, 0], 'prop_root':1, 'reference':'rohf'}
                  - {'num_roots':3, 'follow_root':2, 'reference':'rohf'} - for state-resolved gradients
            * **params["verbosity"]** ( int ): the level of output of the execution-related 
                information [ default : 0]
                  
        full_id ( intList ): the "path" to the Hamiltonian in the Hamiltonian's hierarchy. Usually, 
            this is Py2Cpp_int([0, itraj]) - the index of the trajectory in a swarm of trajectories
            
    Returns:       
        PyObject: obj, with the members:

            * obj.ham_adi ( CMATRIX(nstates,nstates) ): adiabatic Hamiltonian
            * obj.hvib_adi ( CMATRIX(nstates,nstates) ): vibronic Hamiltonian in the adiabatic basis
            * obj.d1ham_adi ( list of ndof CMATRIX(nstates, nstates) objects ): 
                derivatives of the adiabatic Hamiltonian w.r.t. the nuclear coordinate            
 
    """

    # Make a copy of the input parameters dictionary
    params = dict(params_)
    
    # Defaults    
    critical_params = [ "labels" ] 
    default_params = { "nstates":1,
                       "grad_method_gs":"ccsd/sto-3g", 
                       "grad_method_ex":"eom-ccsd/sto-3g", 
                       "charge":0, "spin_multiplicity":1,                       
                       "options":{},
                       "verbosity":0
                     }
    comn.check_input(params, default_params, critical_params)
    
    # Extract the key variables
    grad_method_gs = params["grad_method_gs"]
    grad_method_ex = params["grad_method_ex"]
    charge = params["charge"]
    spin_multiplicity = params["spin_multiplicity"]
    labels = params["labels"]    
    nstates = params["nstates"]
    options = params["options"]
    verbosity = params["verbosity"]
    natoms = len(labels)
    ndof = 3 * natoms
    
    
    obj = tmp()
    obj.ham_adi = CMATRIX(nstates, nstates)    
    obj.hvib_adi = CMATRIX(nstates, nstates)            
    obj.d1ham_adi = CMATRIXList();            
    for idof in range(ndof):        
        obj.d1ham_adi.append( CMATRIX(nstates, nstates) )
  

    Id = Cpp2Py(full_id)
    indx = Id[-1]

    # Setup and execute the PSI4 calculations   
    coords_str = scan.coords2xyz(labels, q, indx)
    
    config = DefaultOption()
    for section in dftpy_config:
        config[section].update(dftpy_config[section])
    #print(config)
    config = OptionFormat(config)
    #print(config)
    dftpy_config, others = ConfigParser(config)
    dftpy_results = OptimizeDensityConf(dftpy_config, others["struct"], others["E_v_Evaluator"])
    
    energy = dftpy_results["energypotential"]["TOTAL"].energy
    forces = dftpy_results["forces"]["TOTAL"]
    
    print("Forces: ", forces)
    
    #print("shape: ",np.shape(forces))
    
                
    for istate in range(nstates):
        
        obj.ham_adi.set(istate, istate, energy * (1.0+0.0j) )
        obj.hvib_adi.set(istate, istate, energy * (1.0+0.0j) )        
        for iatom in range(natoms):        
            obj.d1ham_adi[3 * iatom + 0].set(istate, istate, -forces[iatom, 0] * (1.0+0.0j) )
            obj.d1ham_adi[3 * iatom + 1].set(istate, istate, -forces[iatom, 1] * (1.0+0.0j) )
            obj.d1ham_adi[3 * iatom + 2].set(istate, istate, -forces[iatom, 2] * (1.0+0.0j) )
        
                        
    return obj


In [40]:
%%time 

labels = ["Li", "H"]
q = MATRIX(6,1)
q.set(0,0, 0.0);  q.set(1,0, 0.0); q.set(2,0, 0.0);
q.set(3,0, 0.0);  q.set(4,0, 0.0); q.set(5,0, 1.5);

print( scan.coords2xyz( labels, q, 0) )

Li  0.0 0.0 0.0
H  0.0 0.0 1.5

CPU times: user 0 ns, sys: 958 µs, total: 958 µs
Wall time: 5.78 ms


In [74]:
dftpy_config = {
    "JOB": {"calctype": "Energy Force"},
    "PATH": {"pppath": "/user/kjiang/"},
    "PP": {"Li": "OEPP_lib/OEPP/RECPOT/Li_lda.oe02.recpot",
           "H": "ofpp/DFT-MUEC/h/H.pz-locmodreg_rc0.50-qtp.recpot"},
    "CELL": {"cellfile": "LiH.vasp",
            "elename": "Li H",
            "zval": "1 1",
            "format": "vasp"},
    "EXC": {"xc": "LDA"},
    "KEDF": {"kedf": "vW"}
}

In [83]:
%%time 

params_ = { "labels": labels, "nstates":1, "verbosity":1    }
inp_id = Py2Cpp_int([0, 0])
obj = run_dftpy_adi(q, params_, inp_id, dftpy_config)

The initial grid size is  [79 79 79]
The final grid size is  [80 80 80]
setting key: Li -> /user/kjiang//OEPP_lib/OEPP/RECPOT/Li_lda.oe02.recpot
setting key: H -> /user/kjiang//ofpp/DFT-MUEC/h/H.pz-locmodreg_rc0.50-qtp.recpot
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -7.309475155937E-02     -7.309475E-02   7.277341E-03    1       1       2.698967E-01    
1       -6.592205899926E-01     -5.861258E-01   7.240389E-01    1       5       8.932772E-01    
2       -7.087767212337E-01     -4.955613E-02   4.289131E-01    1       3       1.228931E+00    
3       -7.276106822658E-01     -1.883396E-02   2.574395E-01    1       2       1.458887E+00    
4       -7.510310050099E-01     -2.342032E-02   4.962236E-01    1       2       1.687396E+00    
5       -7.831592339028E-01     -3.212823E-02   5.549774E-01    1       2       1.914863E+00    
6       -8.203141394020E-01     -3.715491E-02   5.241644E-01    1       1       2.040104E+00   